1. [Santander Fast Compact Solution](https://www.kaggle.com/gpreda/santander-fast-compact-solution)
Go to Kaggle website, resgister, attend competition, and make groups.

In [ ]:
import warnings
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import gc 
gc.collect()

In [ ]:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')

In [ ]:
features=train_df.columns[2:]

What Tree is
---
packages required, 

- install [GraphViz](https://www.graphviz.org/download/)
- `graphviz` installed by pip 


To describe what tree is, the following decision Tree demo is created with 4-leaves:


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn import tree
import graphviz

train = train_df.sample(frac=0.8, random_state=2019)
valid = train_df[~train_df.index.isin(train.index) ]
#valid = train.sample(frac=0.2, random_state=42)
#train = train[~train.index.isin(valid.index) ].sample(frac=1)

#model = tree.DecisionTreeClassifier(max_leaf_nodes=4)
model = tree.DecisionTreeRegressor(max_leaf_nodes=4)
model.fit(train[features], train['target'])
#pred_val = model.predict_proba(valid[cols])[:,1]
pred_val = model.predict(valid[features])

print('AUC = ',round( roc_auc_score(valid['target'],pred_val),4 ) )
tree_graph = tree.export_graphviz(model, out_file=None, max_depth = 10,
    impurity = False, feature_names = features, class_names = ['0', '1'],
    rounded = True, filled= True )
graphviz.Source(tree_graph)  

In [ ]:
# about 0.1 probability with target being equal to 1 in train set (size 160000)
sum(train['target'])/160000

In [ ]:
# size of target being 1 in the first split by 'var_81'  
len(train[train['var_81' ]<=11.535])

1. Notice that it finds the "bump" of **var_81** and cuts it off
-  It's second decision is to cut the "bump" off **var_139**. Then it chops **var_12**. These three decisions achieve validation AUC 0.6015. It predicts probability of target=1 to be 0.192, 0.131, 0.166, and 00075 for it's four splits.
- Note that GBMs are a bit different from single decision trees. First, they are additive tree ensemble, i.e. the final prediction is the sum of the prediction of each tree in the model. Number of trees can be in thousands if not more. Second, trees are built one by one by using the gradient of the loss function at the prediction of the previously built trees. Trees are not built using entropy or other single tree metric.

In [ ]:
sns.distplot(train_df[train_df.target==1]['var_81'],  kde=True,bins=200,label='target=1')
sns.distplot(train_df[train_df.target==0]['var_81'],  kde=True,bins=200,label='target=0')
plt.axvline(x=11.535,color='black')
plt.legend()

In [ ]:
gc.collect()

Features
---
1. target, predicted feature
2. var_n, b=0, ...,199, **200** given features
3. new features:
```
      var_200: mean of 200 features
      var_201: std  of 200 eatures
      var_202: mean of largest 10 features
      var_203: mean of smallest 10 features
```  
4. `df.iloc[i,j:]`: the i-th row, and all the columns starting from j-th column (feature)

XGB
---

In [ ]:
import xgboost as xgb

xgb_params = {'eta': 0.01, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.8, 
              'gamma':  5,'lambda':1,'min_child_weight' : 50,  
               'colsample_bylevel' : 0.7,'alpha':0,'booster' : "gbtree",
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True}

features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

In [ ]:
#num_round = 1000000
REPEATED=4

folds = StratifiedKFold(n_splits=REPEATED, shuffle=False, random_state=2019)
oof = np.zeros(len(train_df))
predictions_xgb = np.zeros(len(test_df))

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = xgb.DMatrix(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = xgb.DMatrix(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
    num_round = 5000
    clf_xgb = xgb.train(xgb_params, trn_data, num_round,watchlist,  early_stopping_rounds = 50 ,verbose_eval=100)
    oof[val_idx] = clf_xgb.predict(xgb.DMatrix(train_df.iloc[val_idx][features]), ntree_limit=clf_xgb.best_ntree_limit+50)
    predictions += clf_xgb.predict(xgb.DMatrix(test_df[features]), ntree_limit=xgb_model.best_ntree_limit+50) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof_xgb)))


In [ ]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = predictions
sub.to_csv("output/submission-xgb.csv", index=False)

In [ ]:
import plotly.plotly as py
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks
# offline mode
cufflinks.go_offline()
# Set the global theme for cufflinks
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)

In [ ]:
# 11-16-1
sub.hist()